In [59]:
from task9 import *
from task8 import *
from task7 import *
from task6 import *
from task4 import *
from task3 import *
from copy import deepcopy
from task4 import *
import pytest
from grammars_constants import REGEXP_CFG, GRAMMARS, GRAMMARS_DIFFERENT, CFG_EBNF
from helper import generate_rnd_start_and_final
from rpq_template_test import (
    rpq_cfpq_test,
    different_grammars_test,
    cfpq_algorithm_test,
)
from fixtures import graph
import grammars_constants
from helper import generate_rnd_graph, generate_rnd_dense_graph
import random
from constants import LABELS
import datetime


Проведем сравнение скорости работы алгоритмов КС-достижимости:

In [42]:
time_matrix = datetime.timedelta(0)
time_hellings = datetime.timedelta(0)
time_tensor = datetime.timedelta(0)
time_gll = datetime.timedelta(0)
grammars: list[cfg.CFG] = grammars_constants.GRAMMARS_DIFFERENT

def test_hellings_matrix_tensor_gll(graph, grammar):
    global time_gll, time_matrix, time_tensor, time_hellings
    start_nodes, final_nodes = generate_rnd_start_and_final(graph)
    now1 = datetime.datetime.now()
    cfpq_with_hellings(
        deepcopy(grammar), deepcopy(graph), start_nodes, final_nodes
    )
    now2 = datetime.datetime.now()
    cfpq_with_matrix(
        deepcopy(grammar), deepcopy(graph), start_nodes, final_nodes
    )
    now3 = datetime.datetime.now()
    cfpq_with_tensor(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    now4 = datetime.datetime.now()
    cfpq_with_gll(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    now5 = datetime.datetime.now()
    
    time_hellings += now2 - now1
    time_matrix += now3 - now2
    time_tensor += now4 - now3
    time_gll += now5 - now4
    
    
for i in range(1):
    graph = generate_rnd_dense_graph(1, 100, LABELS)
    for grammar in grammars:
        test_hellings_matrix_tensor_gll(graph, grammar)
    
    
print(f"time hellings {time_hellings}")

print(f"time matrix {time_matrix}")

print(f"time tensor {time_tensor}")

print(f"time gll {time_gll}")
     

[2024-05-22 22:51:45]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x153ca2a50> with n=77, p=0.4, labels=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], choice=<bound method Random.choice of <random.Random object at 0x7f8617817220>>, seed=None


time hellings 0:00:00.249626
time matrix 0:01:10.481834
time tensor 0:00:01.570905
time gll 0:00:00.223763


Из проведенного анализа видно, что хуже всего с задачей КС достижимости справляется матричный алгоритм - он работает на порядок медленнее всех остальных. Для более детаельного анализа остальных алгоритмов, проведем их сравнительный анализ без матричного алгоритма:

In [45]:
time_hellings = datetime.timedelta(0)
time_tensor = datetime.timedelta(0)
time_gll = datetime.timedelta(0)
grammars: list[cfg.CFG] = grammars_constants.GRAMMARS_DIFFERENT

def test_hellings_tensor_gll(graph, grammar):
    global time_gll, time_matrix, time_tensor, time_hellings
    start_nodes, final_nodes = generate_rnd_start_and_final(graph)
    now1 = datetime.datetime.now()
    cfpq_with_hellings(
        deepcopy(grammar), deepcopy(graph), start_nodes, final_nodes
    )
    now2 = datetime.datetime.now()
    cfpq_with_tensor(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    now3 = datetime.datetime.now()
    cfpq_with_gll(
        cfg_to_rsm(deepcopy(grammar)), deepcopy(graph), start_nodes, final_nodes
    )
    now4 = datetime.datetime.now()
    
    time_hellings += now2 - now1
    time_tensor += now3 - now2
    time_gll += now4 - now3
    
    
for i in range(10):
    graph = generate_rnd_dense_graph(1, 100, LABELS)
    for grammar in grammars:
        test_hellings_tensor_gll(graph, grammar)
    
    
print(f"time hellings {time_hellings}")

print(f"time tensor {time_tensor}")

print(f"time gll {time_gll}")

[2024-05-22 22:54:25]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x153ca2e90> with n=18, p=0.4, labels=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], choice=<bound method Random.choice of <random.Random object at 0x7f8617817220>>, seed=None
[2024-05-22 22:54:25]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1536f5f90> with n=100, p=0.4, labels=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], choice=<bound method Random.choice of <random.Random object at 0x7f8617817220>>, seed=None
[2024-05-22 22:54:28]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x153180c50> with n=92, p=0.4, labels=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], choice=<bound method Random.choice of <random.Random object at 0x7f8617817220>>, seed=None
[2024-05-22 22:54:31]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x152c651d0> with n=54, p=0.4,

time hellings 0:00:02.073423
time tensor 0:00:09.431145
time gll 0:00:01.802460


Видим, что алгоритмы Хеллингса и GLL работают примерно с одинаквой скоростью (но алгоритм на основе GLL работает немного быстрее), а тензорный алгоритм от них отстает примерно в 5 раз.

Проведем анализ влияние размера грамматики на время работы тнзорного алгоритма иалгоритма на основе GLL

In [66]:
# ls = ["a", "b"]

smallGrammar = cfg.CFG.from_text("S -> a")

bigGrammar = cfg.CFG.from_text(
                """
                S -> S1 | S2
                S1 -> Sab | S1 a
                Sab -> $ | a Sab b
                S2 -> Sbc | a S2
                Sbc -> $ | b Sbc a
                """
            )
start_nodes, final_nodes = generate_rnd_start_and_final(graph)

graph = generate_rnd_dense_graph(1, 10000, LABELS)

now1 = datetime.datetime.now()
cfpq_with_tensor(
    cfg_to_rsm(deepcopy(smallGrammar)), deepcopy(graph), start_nodes, final_nodes
)
now2 = datetime.datetime.now()
cfpq_with_gll(
    cfg_to_rsm(deepcopy(smallGrammar)), deepcopy(graph), start_nodes, final_nodes
)
now3 = datetime.datetime.now()

timeSmallTensor = now2 - now1
timeSmallGLL = now3 - now2
 
now1 = datetime.datetime.now()
cfpq_with_tensor(
    cfg_to_rsm(deepcopy(bigGrammar)), deepcopy(graph), start_nodes, final_nodes
)
now2 = datetime.datetime.now()
cfpq_with_gll(
    cfg_to_rsm(deepcopy(bigGrammar)), deepcopy(graph), start_nodes, final_nodes
)
now3 = datetime.datetime.now()

timeBigTensor = now2 - now1
timeBigGLL = now3 - now2

print(f"time for small graph with tensor: {timeSmallTensor}")
print(f"time for big graph with tensor: {timeBigTensor}")
print(f"time for small graph with GLL: {timeSmallGLL}")
print(f"time for big graph with GLL: {timeBigGLL}")







AttributeError: 'list' object has no attribute 'CFG'

Видим, что время работы тензорного алгоритма с большой грамматикой оказалось меньше, чем время работы этого же алгоритм, но с маленькой грамматикой из чего можем заулючить, что размерр грамматики несущуственно влияет на время работы данного алгоритма

Время работы алгоритма на осонове GLL с большой грамматикой оказалось 15 процентов больше, чем время работы этоого алгоритма с маленькоой грамматикой.

Из вышесказанного можкм заключить, что размер грамматики в бОльшей степени влияет на время работы алгоритма КС-достижимости на основе GLL, нежели чем тензорного алгоритма

Теперь ответим на второй вопрос. Для этого возьмем regex и CFG, задающие одни и те же ограничения и сравним производительность алгоритма, показавшего наилучшее время работы среди КС-алгоритмов (то есть алгоритма на основе GLL) и сравним ее время работы со врременем работы алгритма из 4-ой домашки, то есть с алгоритмом регулярной достижимости на основе BFS

In [62]:
regex_grammar = grammars_constants.REGEXP_CFG
time_regex = datetime.timedelta(0)
time_gll = datetime.timedelta(0)

def run_algos(graph, regex, cfg):
    global time_regex, time_gll
    start_nodes, final_nodes = generate_rnd_start_and_final(graph.copy())
    fa = FiniteAutomaton(
            graph_to_nfa(deepcopy(graph), deepcopy(start_nodes), deepcopy(final_nodes))
    )
    constraint_fa = FiniteAutomaton(regex_to_dfa(regex))
    now1 = datetime.datetime.now()
    reachability_with_constraints(deepcopy(fa), deepcopy(constraint_fa))
    now2 = datetime.datetime.now()
    cfpq_with_gll(
        cfg_to_rsm(deepcopy(cfg)), deepcopy(graph), start_nodes, final_nodes
    )
    now3 = datetime.datetime.now()
    time_regex += now2-now1
    time_gll += now3-now2
    
    
    
for i in range(1):
    graph = generate_rnd_dense_graph(1, 100, LABELS)
    for (regex, cfg_list) in regex_grammar:
        if len(cfg_list) == 0:
            continue
        run_algos(graph, regex, cfg_list[0])
        
print(f"time_regex = {time_regex}")
print(f"time_gll = {time_gll}")

    
    

[2024-05-22 23:39:12]>INFO>[GNP] Create a Erdős-Rényi graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x156e47190> with n=91, p=0.4, labels=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], choice=<bound method Random.choice of <random.Random object at 0x7f8617817220>>, seed=None


time_regex = 0:00:09.585413
time_gll = 0:00:00.211527


Видим, что время работы алгоритма GLL на порядок меньше, чем время работы алгоритма регулярной достижимости на основе BFS. Это значит, что смысл исползоват алгоритмы для КС ограничений для решения задач регулярной достижимости есть - особенно, когда у нас уже есть CFG для данного регулярного выражения.

ВЫВОДЫ:
1. Какой из трёх указанных алгоритмов обладает лучшей производительностью? GLL
2. Имеет ли смысл для решения задачи достижимости с регулярными ограничениями использовать алгоритмы для КС ограничений (ведь регулярные --- частный случай КС) или всё же лучше использовать специализированные алгоритмы для регулярных ограничений? ДА
3. Как влияет грамматика на производительность тензорного алгоритма и алгоритма на основе GLL? Если зафиксировать язык, то как свойства грамматики (размер, (не)однозначность) влияют на производительность.
Размер грамматики влияет на время работы этих алгоритмов не очень сильно, но при этом это влияние больше на алгоритм на основе GLL.